# DI Automated Scripts
This notebook extracts metadata from video files 
<br>
<br>
![UofC logo](../assets/images/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import pandas as pd
from dotenv import load_dotenv

C:\Users\chris\AppData\Local\Temp\ipykernel_28760\1769867177.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# reload for module caching
from importlib import reload
import src.video_utilities
reload(src.video_utilities)
import src.id_utilities
reload(src.id_utilities)
import src.reader_utilities
reload(src.reader_utilities)
import src.export_utilities
reload(src.export_utilities)
import src.json_utilities
reload(src.json_utilities)

# import custom functions
from src.video_utilities import add_video_properties
from src.id_utilities import add_json_index
from src.reader_utilities import load_json
from src.export_utilities import export_to_json, export_to_excel
from src.json_utilities import restructure_metadata

In [3]:
records_folder = "../records" 
json_dir = "../records/JSON/all_data"
curr_dir = os.getcwd()

In [4]:
# load root directory
dotenv_path = os.path.join(curr_dir, ".env")
load_dotenv(dotenv_path)
root_path = os.getenv("ROOT_FOLDER")
input(f"Is this the right directory - {root_path}?")

''

## Run functions

In [5]:
""" load JSON files """
metadata_rgb = load_json(json_dir, "rgb.json")

In [6]:
""" extract and export metadata from video (rgb) """
json_rgb_filename = json_dir + "/rgb_complete.json"
complete_metadata_rgb = add_video_properties(root_path, metadata_rgb, "rgb")
export_to_json(complete_metadata_rgb, json_rgb_filename)

ZeroDivisionError occurred while processing E:/DI_centre_structuredD:/DI_centre_structured/DI_THERMAL_FLIR//1\2 meters\H\WOB\frames_1_H_WOB_2m_2_lower_bound\1_2-2m-WOB-HB_0.png: float division by zero


KeyError: 'ID'

In [ ]:
""" restructure metadata"""
restructured_complete_metadata_rgb = restructure_metadata(complete_metadata_rgb)
json_rgb_filename = json_dir + "/rgb_metadata.json"
export_to_json(restructured_complete_metadata_rgb, json_rgb_filename)

Data successfully exported to ../records/JSON/all_data/rgb_metadata.json


In [ ]:
""" patient info to pandas DataFrame (rgb)"""
rgb_xlsx = "../records/xlsx/rgb.xlsx"
complete_metadata_rgb_with_index = add_json_index(complete_metadata_rgb)
rgb_df = export_to_excel(complete_metadata_rgb_with_index, rgb_xlsx)

Data successfully exported to ../records/xlsx/rgb.xlsx


In [ ]:
""" read exported excel (rgb) """
new_rgb_df = pd.read_excel(rgb_xlsx)
new_rgb_df.describe()

,frames,length,old fps,alias
count,6.000000,6.000000,6.000000,6.0
mean,1704.333333,44.284740,35.083572,15.0
std,1263.451569,24.791055,13.586104,0.0
min,143.000000,7.703175,18.563774,15.0
25%,1137.250000,36.633006,29.402381,15.0
50%,1464.000000,38.394212,29.627625,15.0
75%,2057.000000,63.685860,44.617079,15.0
max,3872.000000,73.054497,53.741547,15.0
